In [1]:
from PIL import Image,ImageEnhance,ImageFilter
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import preprocessing
import math
import string

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras import backend as K

Using TensorFlow backend.


## 读入图片

In [2]:
# 将文件名称加入文件名队列
filename = []
for i in range(10000):
    filename.append('../dataset/train/'+str(i)+'.jpg')

# 读入图片
dataset = np.zeros(shape=(10000,80,100))  # 初始化图片存储数组
for index,img in enumerate(filename):
    im = Image.open(img).convert('L')  # 将图片灰度化读入
    im = im.resize((100,80),Image.ANTIALIAS)  # 将图片缩放为相同大小（100*80）
    enhancer = ImageEnhance.Contrast(im) #增加对比对
    im = enhancer.enhance(2)
    im = np.asarray(im, dtype=np.int64)
    dataset[index] = im

# 将图片灰度转换后，重新转化形状（单通道图片也需要单独一维）
dataset = dataset.reshape(-1,80,100,1) 

## 读入标签

In [3]:
# 从文件中读入标签
img_label = pd.read_csv('../dataset/train_labels.csv')
img_label = img_label['y'].values.astype(str) # 和使用np.string_类型不同，待确定

# 将标签转化为str类型，补0
for index,label in enumerate(img_label):
    if len(label) == 1:
        img_label[index] = str('00')+label
    elif len(label) == 2:
        img_label[index] = str('0')+label


## 将标签进行编码

In [4]:
labeldict = {'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9} # 定义标签字典
num_classes = 10  # 确定数字类别数（共10类）

# 将标签的每一个字符进行onehot-encoding
y = []
for i in range(len(img_label)):
    c0 = keras.utils.to_categorical(labeldict[img_label[i][0]], num_classes)
    c1 = keras.utils.to_categorical(labeldict[img_label[i][1]], num_classes)
    c2 = keras.utils.to_categorical(labeldict[img_label[i][2]], num_classes)
    c = np.concatenate((c0,c1,c2)) # 将三个onehot-encoding后的标签合并
    y.append(c)
y = np.array(y)
X = dataset

## 划分验证集与训练集

In [5]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.1,random_state = 0)

## 整理channels

In [6]:
# 由于keras调用tensorflow和Theano后端在池化层使用的通道顺序不同，需要分别对其进行转化

img_rows = 80
img_cols = 100
print(K.image_data_format())
print(x_train.shape,x_test.shape)

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

channels_last
(9000, 80, 100, 1) (1000, 80, 100, 1)
(9000, 80, 100, 1) (9000, 30)
(1000, 80, 100, 1) (1000, 30)


## 像素值归一化处理

In [7]:
x_train = 255 - x_train
x_test = 255 - x_test
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [34]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (9000, 80, 100, 1)
9000 train samples
1000 test samples


## 设计神经网络模型

In [62]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())          
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(num_classes*3, activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


In [63]:
batch_size = 32
epochs = 100

# 设置early_stopping回调函数，当验证集acc不再升高时停止训练
early_stopping = EarlyStopping(monitor='val_acc', patience=10, verbose=2,mode='max')

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),callbacks=[early_stopping])

Train on 9000 samples, validate on 1000 samples
Epoch 1/100
9000/9000 [==============================] - 14s 2ms/step - loss: 0.2871 - acc: 0.9065 - val_loss: 0.1484 - val_acc: 0.9444
Epoch 2/100
9000/9000 [==============================] - 12s 1ms/step - loss: 0.0919 - acc: 0.9674 - val_loss: 0.0710 - val_acc: 0.9743
Epoch 3/100
9000/9000 [==============================] - 12s 1ms/step - loss: 0.0471 - acc: 0.9832 - val_loss: 0.0553 - val_acc: 0.9799
Epoch 4/100
9000/9000 [==============================] - 12s 1ms/step - loss: 0.0277 - acc: 0.9902 - val_loss: 0.0382 - val_acc: 0.9872
Epoch 5/100
9000/9000 [==============================] - 13s 1ms/step - loss: 0.0170 - acc: 0.9939 - val_loss: 0.0313 - val_acc: 0.9899
Epoch 6/100
9000/9000 [==============================] - 12s 1ms/step - loss: 0.0118 - acc: 0.9958 - val_loss: 0.0292 - val_acc: 0.9901
Epoch 7/100
9000/9000 [==============================] - 12s 1ms/step - loss: 0.0087 - acc: 0.9969 - val_loss: 0.0205 - val_acc: 0.9936


In [64]:
# 保存模型
model.save('my_model milestone 3-12.h5')

## 读入测试集

In [65]:
filename = []
for i in range(20000):
    filename.append('../dataset/test/'+str(i)+'.jpg')
test_dataset = np.zeros(shape=(20000,80,100))  # 存储图片的数组
for index,img in enumerate(filename):
    im = Image.open(img).convert('L')  # 将图片灰度化读入
    im = im.resize((100,80),Image.ANTIALIAS)  # 将图片缩放为相同大小（100*80）
    enhancer = ImageEnhance.Contrast(im) #增加对比对
    im = enhancer.enhance(2)
    im = np.asarray(im, dtype=np.int64)
    test_dataset[index] = im


test_dataset = test_dataset.reshape(-1,80,100,1) # 将图片灰度转换

## 测试集预处理

In [66]:
img_rows = 80
img_cols = 100
print(K.image_data_format())
print(test_dataset.shape)

if K.image_data_format() == 'channels_first':
    test_dataset = test_dataset.reshape(test_dataset.shape[0], 1, img_rows, img_cols)
else:
    test_dataset = test_dataset.reshape(test_dataset.shape[0], img_rows, img_cols, 1)

test_dataset = 255 - test_dataset
test_dataset = test_dataset.astype('float32')
test_dataset /= 255

channels_last
(20000, 80, 100, 1)


## 使用模型对测试集进行预测

In [67]:
pred = model.predict(test_dataset,batch_size = 32)

In [68]:
outdict = ['0','1','2','3','4','5','6','7','8','9']
pred_result = []
for i in range(pred.shape[0]):
    c0 = outdict[np.argmax(pred[i][:10])]
    c1 = outdict[np.argmax(pred[i][10:20])]
    c2 = outdict[np.argmax(pred[i][20:30])]
    c = c0+c1+c2
    pred_result.append(c)

In [69]:
pred_result = np.array(pred_result).astype('int')

In [70]:
pred_result = pd.DataFrame(pred_result,dtype='int64')
pred_result.columns = ['y']

# test_pre
test_id = pd.DataFrame(np.arange(20000))
test_id.columns = ['id']

test_id['y'] = pred_result['y']
test_id

test_id.to_csv('milestone3-12 Nov 24.csv',index=0)

In [71]:
from sklearn.metrics import accuracy_score
x = pd.read_csv('../dataset/corr_result.csv')
x = x['y']
x = np.array(x)
x = x[:4000]

y = pd.read_csv('./milestone3-12 Nov 24.csv')
y = y['y']
y = np.array(y)
y = y[:4000]

acc = accuracy_score(x,y)
acc

0.8355